<a href="https://colab.research.google.com/github/theMADAIguy/STOCK-PREDICTION-USING-SVM-REGRESSION/blob/master/pytorch_tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms

In [0]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [43]:
trainSet=torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
testSet=torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
trainLoader=torch.utils.data.DataLoader(trainSet, batch_size=4, shuffle=True, num_workers=2)
testLoader=torch.utils.data.DataLoader(testSet, batch_size=4, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [47]:
for epoch in range(20):
  running_loss=0.0
  for i, data in enumerate(trainLoader,0):
    input, labels=data

    optimizer.zero_grad()
    output=net(input)
    loss=criterion(output, labels)
    loss.backward()
    optimizer.step()

    running_loss=running_loss+loss.item()
    if i % 2000 == 1999:
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 2000))
      running_loss = 0.0


[1,  2000] loss: 2.235
[1,  4000] loss: 1.851
[1,  6000] loss: 1.653
[1,  8000] loss: 1.566
[1, 10000] loss: 1.523
[1, 12000] loss: 1.452
[2,  2000] loss: 1.373
[2,  4000] loss: 1.356
[2,  6000] loss: 1.344
[2,  8000] loss: 1.303
[2, 10000] loss: 1.289
[2, 12000] loss: 1.244
[3,  2000] loss: 1.206
[3,  4000] loss: 1.199
[3,  6000] loss: 1.184
[3,  8000] loss: 1.199
[3, 10000] loss: 1.177
[3, 12000] loss: 1.176
[4,  2000] loss: 1.066
[4,  4000] loss: 1.107
[4,  6000] loss: 1.099
[4,  8000] loss: 1.109
[4, 10000] loss: 1.123
[4, 12000] loss: 1.118
[5,  2000] loss: 1.020
[5,  4000] loss: 1.021
[5,  6000] loss: 1.041
[5,  8000] loss: 1.050
[5, 10000] loss: 1.044
[5, 12000] loss: 1.049
[6,  2000] loss: 0.951
[6,  4000] loss: 0.982
[6,  6000] loss: 0.970
[6,  8000] loss: 0.986
[6, 10000] loss: 1.003
[6, 12000] loss: 0.992
[7,  2000] loss: 0.904
[7,  4000] loss: 0.912
[7,  6000] loss: 0.955
[7,  8000] loss: 0.947
[7, 10000] loss: 0.946
[7, 12000] loss: 0.980
[8,  2000] loss: 0.871
[8,  4000] 

In [53]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/
PATH = './cifar_net.pth'
import torch
torch.save(net.state_dict(), PATH)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [54]:
##Testing


net2=Net()
net2.load_state_dict(torch.load(PATH))
print(net2)


dataiter=iter(testLoader)
images, labels=dataiter.next()
print(labels)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
tensor([3, 8, 8, 0])


In [55]:
test_outputs=net2(images)
_, predicted = torch.max(test_outputs,1)
print(predicted)

tensor([3, 8, 8, 8])


In [56]:
correct=0
total=0

with torch.no_grad():
  for data in testLoader:
    images, labels = data
    test_outputs=net2(images)
    _, predicted = torch.max(test_outputs.data,1)
    total=total+labels.size(0)
    correct+=(predicted==labels).sum().item()
  print(total)
  print(correct)

10000
6037


In [57]:
print("Accuracy:", (correct*100/total))

Accuracy: 60.37
